In [139]:
import pandas as pd

In [ ]:
# converte o arquivo xlsx para csv, para otimizar a consulta no Pandas
ano = 2017
file_name = f'OD_{ano}.xlsx'
df = pd.read_excel(file_name)
file_name_csv = file_name.replace('.xlsx', '.csv')
df.to_csv(file_name_csv, index=False)
print('Arquivo csv salvo com sucesso.')

# converte o arquivo xlsx para csv, para otimizar a consulta no Pandas
df = pd.read_csv(file_name_csv, low_memory=False)
df.info()

In [ ]:
file_name = 'projecoes_2018_populacao_2010_2060_20200406.xls'
df_ibge = pd.read_excel(file_name, sheet_name='BRASIL', skiprows=50, skipfooter=220)
# remove coluna desnecessária
del df_ibge['Unnamed: 52']
# remove as golunas desnecessárias (2010 até 2016)
for coluna_a_remover in range(2010, 2017):
    del df_ibge[coluna_a_remover]
len(df_ibge)
print(df_ibge.columns)
df_ibge.tail()

In [ ]:
df_faixas_etarias = df_ibge['GRUPO ETÁRIO']
# converte a Series em dataframe
df_faixas_etarias = df_faixas_etarias.to_frame()
# remove última linha, que é NA
df_faixas_etarias = df_faixas_etarias.dropna()
# Remove a primeira linha, que é "Total"
df_faixas_etarias = df_faixas_etarias[1:]
df_faixas_etarias

In [ ]:
# cria as colunas que serão usadas para classificar as pessoas dentro do range
df_faixas_etarias[['idade_minima', 'idade_maxima']] = df_faixas_etarias['GRUPO ETÁRIO'].str.split('-', expand = True)
# ajusta a idade_maxima dos 90+
df_faixas_etarias['idade_maxima'].fillna(150, inplace=True)
# remove o sinal de + para a idade de 90 anos
df_faixas_etarias['idade_minima'] = df_faixas_etarias['idade_minima'].str.replace('+', '', regex=False)
df_faixas_etarias.rename(columns={'GRUPO ETÁRIO': 'grupo_etario'}, inplace=True)
# altera os campos para numéricos
df_faixas_etarias['idade_minima'] = pd.to_numeric(df_faixas_etarias['idade_minima'])
df_faixas_etarias['idade_maxima'] = pd.to_numeric(df_faixas_etarias['idade_maxima'])
df_faixas_etarias

In [ ]:
# a partir dos grupos etários da lista do ibge, cria a coluna 
# grupo_etario no dataframe OD2017, incluindo o seu grupo etário
grupos = [
    0,
    4,
    9,
    14,
    19,
    24,
    29,
    34,
    39,
    44,
    49,
    54,
    59,
    64,
    69,
    74,
    79,
    84,
    89,
    150
]
labels = df_faixas_etarias['grupo_etario'].to_list()
df['grupo_etario'] = pd.cut(df['IDADE - Idade'], bins=grupos, include_lowest=False, precision=0, labels=labels)
df[['IDADE - Idade', 'grupo_etario']].tail()
df[['ID_PESS - Identifica Pessoa', 'IDADE - Idade', 'grupo_etario']].head()

In [ ]:
# cria uma cópia apenas dos campos que serão usados para fazer o 'JOIN' 
# com o dataframe que já foi lido no Power BI
df_saida = df[['ID_PESS - Identifica Pessoa', 'IDADE - Idade', 'grupo_etario']].copy()
a_renomear = {
    'ID_PESS - Identifica Pessoa': 'ID_PESS',
    'IDADE - Idade': 'IDADE',
    'grupo_etario': 'GRUPO ETÁRIO'
}
df_saida.rename(columns=a_renomear, inplace=True)
df_saida.head(10)

In [ ]:
# faz o merge dos dados do IBGE com o df_saida
df_saida_ibge = df_saida.merge(df_ibge)
# remove registros duplicados, se houver
print('Antes de remover duplicados ->', len(df_saida_ibge))
df_saida_ibge.drop_duplicates(inplace=True)
print('Após remover duplicados ->', len(df_saida_ibge))
df_saida_ibge.head()

In [ ]:
df_saida_ibge.head()

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
# cria algumas colunas com o fator de evolução, no formato
# FATOR_ANO_FINAL_ANO_INICIAL
ano_inicial, ano_final = 2017, 2023
df_saida_ibge[f'FATOR_{ano_inicial}_{ano_final}'] = (df_saida_ibge[ano_final]/df_saida_ibge[ano_inicial])

ano_inicial, ano_final = 2017, 2025
df_saida_ibge[f'FATOR_{ano_inicial}_{ano_final}'] = (df_saida_ibge[ano_final]/df_saida_ibge[ano_inicial])

ano_inicial, ano_final = 2017, 2035
df_saida_ibge[f'FATOR_{ano_inicial}_{ano_final}'] = (df_saida_ibge[ano_final]/df_saida_ibge[ano_inicial])

ano_inicial, ano_final = 2017, 2040
df_saida_ibge[f'FATOR_{ano_inicial}_{ano_final}'] = (df_saida_ibge[ano_final]/df_saida_ibge[ano_inicial])

ano_inicial, ano_final = 2017, 2045
df_saida_ibge[f'FATOR_{ano_inicial}_{ano_final}'] = (df_saida_ibge[ano_final]/df_saida_ibge[ano_inicial])

ano_inicial, ano_final = 2017, 2050
df_saida_ibge[f'FATOR_{ano_inicial}_{ano_final}'] = (df_saida_ibge[ano_final]/df_saida_ibge[ano_inicial])

ano_inicial, ano_final = 2017, 2055
df_saida_ibge[f'FATOR_{ano_inicial}_{ano_final}'] = (df_saida_ibge[ano_final]/df_saida_ibge[ano_inicial])

ano_inicial, ano_final = 2017, 2060
df_saida_ibge[f'FATOR_{ano_inicial}_{ano_final}'] = (df_saida_ibge[ano_final]/df_saida_ibge[ano_inicial])

df_saida_ibge.head()

In [ ]:
df_saida_ibge.columns

In [ ]:
file_path_saida = 'OD_2017_fator_de_comparação_IBGE.xlsx'
df_saida_ibge.to_excel(file_path_saida, index=False)
print('Arquivo de saída salvo com sucesso.')